# Decision Tree
---

In [ ]:
import numpy as np
import pandas as pd
from IPython.display import display

### Data Preprocessing

Importing the data from the UCI ML Repository using code.

In [11]:
%pip install ucimlrepo

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: C:\Users\skuma\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [25]:
from ucimlrepo import fetch_ucirepo
wine_quality = fetch_ucirepo(id=186)
X = wine_quality.data.features  
y = wine_quality.data.targets   

In [26]:
X.shape, y.shape

((6497, 11), (6497, 1))

In [27]:
display(X.head())

,fixed_acidity,volatile_acidity,citric_acid,residual_sugar,chlorides,free_sulfur_dioxide,total_sulfur_dioxide,density,pH,sulphates,alcohol
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4


In [28]:
display(y.head())

,quality
0,5
1,5
2,5
3,6
4,5


In [29]:
# Convert to NumPy
xFeat = X.to_numpy()
y = y.to_numpy()

# Convert quality scores into binary classification
y = np.where(y >= 6, 1, 0).reshape(-1, 1)  # Good quality (1) if ≥6, else Bad (0)

In [30]:
xFeat.shape, y.shape

((6497, 11), (6497, 1))

Split into test and train

In [31]:
# Random seed
np.random.seed(42)
# Split into training (80%) and testing (20%)
indices = np.random.permutation(len(xFeat))
split_idx = int(0.8 * len(xFeat))
train_idx, test_idx = indices[:split_idx], indices[split_idx:]

xTrain, yTrain = xFeat[train_idx], y[train_idx]
xTest, yTest = xFeat[test_idx], y[test_idx]

Now the data is ready for model training.

## Decision Tree

Define the model

In [32]:

# Gini impurity calculation
def gini(y):
    _ , counts = np.unique(y, return_counts=True)
    return 1 - np.sum((counts / counts.sum()) ** 2)

# Find the best feature and threshold to split on
def find_best_split(xFeat, y):
    best_gain = 0
    best_feature = None
    best_threshold = None
    n_samples, n_features = xFeat.shape

    current_gini = gini(y)

    for feature in range(n_features):
        thresholds = np.unique(xFeat[:, feature])  # Try unique values as thresholds

        for threshold in thresholds:
            left_idx = xFeat[:, feature] <= threshold
            right_idx = xFeat[:, feature] > threshold

            if left_idx.sum() == 0 or right_idx.sum() == 0:
                continue  # Skip if a split results in empty nodes

            left_gini = gini(y[left_idx])
            right_gini = gini(y[right_idx])
            weighted_gini = (left_idx.sum() / n_samples) * left_gini + \
                            (right_idx.sum() / n_samples) * right_gini

            gain = current_gini - weighted_gini  # Information gain

            if gain > best_gain:
                best_gain, best_feature, best_threshold = gain, feature, threshold

    return best_feature, best_threshold

In [33]:
# Node structure
class Node:
    def __init__(self, feature=None, threshold=None, left=None, right=None, prediction=None):
        self.feature = feature
        self.threshold = threshold
        self.left = left
        self.right = right
        self.prediction = prediction  # Only set for leaf nodes

In [34]:
# Train function
class DecisionTreeClassifier:
    def __init__(self):
        self.tree = None
        self.depth = 0
        self.max_depth = 10
        self.min_samples = 5

    # (a) Define the train function using xFeat and y as parameters.
    def train(self, xFeat, y):
        # If all labels are the same, return a leaf node
        if len(np.unique(y)) == 1:
            return Node(prediction=y[0][0])

        # If stopping conditions are met, return a leaf node with the most common class
        if self.depth >= self.max_depth or len(y) < self.min_samples:
            most_common_label = np.bincount(y.flatten()).argmax()
            return Node(prediction=most_common_label)

        # Find the best feature and threshold to split the data
        best_feature, best_threshold = find_best_split(xFeat, y)

        # If no valid split is found, return a leaf node
        if best_feature is None:
            most_common_label = np.bincount(y.flatten()).argmax()
            return Node(prediction=most_common_label)

        # Split the data
        left_idx = xFeat[:, best_feature] <= best_threshold
        right_idx = xFeat[:, best_feature] > best_threshold

        # Recursively build left and right subtrees
        self.depth += 1
        left_subtree = self.train(xFeat[left_idx], y[left_idx])
        right_subtree = self.train(xFeat[right_idx], y[right_idx])

        # Return a node with the best feature, threshold, and subtrees
        self.tree = Node(feature=best_feature, threshold=best_threshold, 
                         left=left_subtree, right=right_subtree)
        
        
        return self.tree  # Important: Return the root node


    # (b) Predict function
    def _predict_single(self, row, node):
        """Traverse the tree to classify a single sample."""
        if node.prediction is not None:
            return node.prediction

        if row[node.feature] <= node.threshold:
            return self._predict_single(row, node.left)
        else:
            return self._predict_single(row, node.right)

    def predict(self, xTest):
        """Predict classes for multiple samples."""
        return np.array([self._predict_single(row, self.tree) for row in xTest]).reshape(-1, 1)


In [35]:
# Train the Decision Tree Model
dt = DecisionTreeClassifier()
dt.train(xTrain, yTrain)

# Predict on Test Data
yPred = dt.predict(xTest)
yTrainPred = dt.predict(xTrain)

# Compute Accuracy
accuracy = np.mean(yPred == yTest)
train_accuracy = np.mean(yTrainPred == yTrain)  
print(f"Train Accuracy: {train_accuracy * 100:.2f}%")
print(f"Test Accuracy: {accuracy * 100:.2f}%")

Train Accuracy: 73.97%
Test Accuracy: 73.23%


## Question 2

(a)  Holdout Method

(b) K-folds cross validation